In [25]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [26]:
df = pd.read_csv('Fin Machine Learning Ready.csv')
df = df.fillna('missing')
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Salary_Data,Max_Salary,Min_Salary,Average_Salary,Age of the company in years,AWS,Modeling,Equity,SQL,Fintech,python,Risk,Consulting,Excel,Type,Seniority_Status
0,7,Analyst,£28K-£42K (Glassdoor Est.),We are always looking for exceptional talent t...,3.4,Fahrenheit 212,"London, England","New York, NY",51 to 200 employees,2006,Company - Private,Consulting,Business Services,£10 to £25 million (GBP),missing,28000-42000,28000,42000,35000.0,14,0,1,0,0,0,0,0,0,1,Unspesified Analyst,Unspesified Title
1,11,Commodities Analyst,£53K-£96K (Glassdoor Est.),Job Description\n\nCommodities Analysts work c...,5,Citadel,"London, England","Chicago, IL",1001 to 5000 employees,1990,Company - Private,missing,missing,£50 to £100 million (GBP),"Soros Fund Management, D. E. Shaw & Co. - Inve...",53000-96000,53000,96000,74500.0,30,0,1,0,0,0,1,1,0,1,Unspesified Analyst,Unspesified Title
2,16,B2B Analyst,£27K-£52K (Glassdoor Est.),The Company\n\nAddison Lee Group provides seam...,2.2,Addison Lee,"Paddington, England","London, United Kingdom",1001 to 5000 employees,1975,Company - Private,Haulage,Transportation & Logistics,£100 to £500 million (GBP),missing,27000-52000,27000,52000,39500.0,45,0,0,0,0,0,0,1,0,0,Unspesified Analyst,Unspesified Title
3,19,Equity Financial Analyst,£36K-£65K (Glassdoor Est.),A Career in Long/Short Equities at Point72\nLo...,3.9,Point72,"London, England","Stamford, CT",1001 to 5000 employees,2014,Company - Private,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable,missing,36000-65000,36000,65000,50500.0,6,0,1,1,0,0,0,1,0,0,financial analyst,Unspesified Title
4,29,"Business Analyst, UK",£33K-£35K (Glassdoor Est.),We are currently seeking a self-motivated rese...,4.3,Lifescience Dynamics,"London, England","London, United Kingdom",1 to 50 employees,2004,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Unknown / Non-Applicable,missing,33000-35000,33000,35000,34000.0,16,0,0,0,0,0,0,0,1,1,business analyst,Unspesified Title


In [27]:
df.shape

(419, 31)

In [28]:
df.columns


Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'Salary_Data', 'Max_Salary', 'Min_Salary', 'Average_Salary',
       'Age of the company in years', 'AWS', 'Modeling', 'Equity', 'SQL',
       'Fintech', 'python', 'Risk', 'Consulting', 'Excel', 'Type',
       'Seniority_Status'],
      dtype='object')

In [29]:
# Creating values for our comparative model
df_model = df[['Rating', 'Size', 'Age of the company in years', 'Industry', 'Sector', 'Revenue',
                         'Type '
                         'of '
                         'ownership', 'Risk', 'SQL', 'Excel', 'python', 'Seniority_Status', 'Average_Salary']]


In [30]:
df_model.head()

,Rating,Size,Age of the company in years,Industry,Sector,Revenue,Type of ownership,Risk,SQL,Excel,python,Seniority_Status,Average_Salary
0,3.4,51 to 200 employees,14,Consulting,Business Services,£10 to £25 million (GBP),Company - Private,0,0,1,0,Unspesified Title,35000.0
1,5,1001 to 5000 employees,30,missing,missing,£50 to £100 million (GBP),Company - Private,1,0,1,1,Unspesified Title,74500.0
2,2.2,1001 to 5000 employees,45,Haulage,Transportation & Logistics,£100 to £500 million (GBP),Company - Private,1,0,0,0,Unspesified Title,39500.0
3,3.9,1001 to 5000 employees,6,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable,Company - Private,1,0,0,0,Unspesified Title,50500.0
4,4.3,1 to 50 employees,16,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Unknown / Non-Applicable,Company - Private,0,0,1,0,Unspesified Title,34000.0


In [31]:
# creating dummy data( 1 if true 0 if false)
df_dum = pd.get_dummies(df_model, drop_first = True) 
df_dum

,Risk,SQL,Excel,python,Average_Salary,Rating_2.1,Rating_2.2,Rating_2.4,Rating_2.5,Rating_2.6,Rating_3.1,Rating_3.2,Rating_3.3,Rating_3.4,Rating_3.5,Rating_3.6,Rating_3.7,Rating_3.8,Rating_3.9,Rating_4.0,Rating_4.1,Rating_4.2,Rating_4.3,Rating_4.4,Rating_4.5,Rating_4.8,Rating_5.0,Rating_missing,Size_10000+ employees,Size_1001 to 5000 employees,Size_201 to 500 employees,Size_5001 to 10000 employees,Size_501 to 1000 employees,Size_51 to 200 employees,Size_Unknown,Age of the company in years_3.0,Age of the company in years_4.0,Age of the company in years_6.0,Age of the company in years_10.0,Age of the company in years_11.0,...,Industry_Insurance Operators,Industry_Internet,Industry_Investment Banking & Asset Management,Industry_Legal,Industry_Lending,Industry_Logistics & Supply Chain,Industry_Oil & Gas Services,Industry_Publishing,Industry_Real Estate,Industry_Staffing & Outsourcing,Industry_missing,Sector_Accounting & Legal,Sector_Aerospace & Defence,Sector_Biotech & Pharmaceuticals,Sector_Business Services,Sector_Finance,Sector_Information Technology,Sector_Insurance,Sector_Manufacturing,Sector_Media,"Sector_Oil, Gas, Energy & Utilities",Sector_Real Estate,Sector_Retail,Sector_Transportation & Logistics,Sector_missing,Revenue_£1 to £2 billion (GBP),Revenue_£10 to £25 million (GBP),Revenue_£10+ billion (GBP),Revenue_£100 to £500 million (GBP),Revenue_£2 to £5 billion (GBP),Revenue_£25 to £50 million (GBP),Revenue_£5 to £10 billion (GBP),Revenue_£50 to £100 million (GBP),Revenue_£500 million to £1 billion (GBP),Type of ownership_Company - Public,Type of ownership_Other Organisation,Type of ownership_Private Practice / Firm,Type of ownership_Subsidiary or Business Segment,Seniority_Status_Unspesified Title,Seniority_Status_Vice President
0,0,0,1,0,35000.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,0,1,1,74500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
2,1,0,0,0,39500.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,50500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,34000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,0,0,1,0,54500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
415,1,1,1,1,84000.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
416,1,0,1,0,47000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
417,0,0,1,0,21000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [32]:
from sklearn.model_selection import train_test_split

X = df_dum.drop('Average_Salary', axis = 1) #create instance of data with no salaray
y = df_dum.Average_Salary.values # instance of data with just salary data as a series
X_train, X_test, y_train, y_test = train = train_test_split(X, y, test_size = .15, random_state = 42)

In [33]:
#Model Evaluation
def evaluate(model, X_test, Y_test):
    predictions = model.predict(X_train)
    errors = abs(predictions - y_train)
    mape = 100 * np.mean(errors / y_train)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: off by ${:0.4f} .'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

# From Sklearn ensemble libraries:

# Random Forest Classifier

In [34]:
from sklearn.model_selection import cross_val_score

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [36]:
rfc = RandomForestClassifier()

In [37]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [38]:
rfc.score(X_test, y_test)

0.8888888888888888

In [39]:
y_pred = rfc.predict(X_test)

# GridSearch CV parameters

In [40]:
from sklearn.model_selection import GridSearchCV

In [41]:
parameters = {
    'n_estimators'      : [250,500,1000,1250,1500],
    'max_depth'         : [8, 9, 10, 11, 12, 13, 14, 15, 17],
    'random_state'      : [0],
    #'max_features': ['auto'],
    #'criterion' :['gini']
}

In [42]:
gs_rfc = GridSearchCV(rfc, parameters , cv=10, n_jobs=-1, verbose = 2)
gs_rfc.fit(X_train, y_train)

Fitting 10 folds for each of 45 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   38.7s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  9.6min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rand

In [43]:
print(gs_rfc.best_params_)

{'max_depth': 10, 'n_estimators': 250, 'random_state': 0}


In [44]:
best_grid = gs_rfc.best_estimator_
grid_accuracy = evaluate(best_grid, X_train, y_train)

Model Performance
Average Error: off by $198.0337 .
Accuracy = 99.51%.


# Random Forest Regressor

In [45]:
from sklearn.ensemble import RandomForestRegressor

In [46]:
rfr = RandomForestRegressor()

In [47]:
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [48]:
rfr.score(X_test, y_test)

0.9359017420145366

In [49]:
from sklearn.model_selection import cross_validate

In [50]:
cv = cross_validate(rfc, X, y, cv=3)

In [51]:
cv['test_score'].mean()

0.8662555669749915

In [52]:
# GridSearch

In [53]:
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

In [54]:
grid_search = GridSearchCV(estimator = rfr, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [55]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 576 candidates, totalling 1728 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 1728 out of 1728 | elapsed: 10.1min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [56]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 90,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 1000}

In [57]:
def evaluate(model, X_test, Y_test):
    predictions = model.predict(X_train)
    errors = abs(predictions - y_train)
    mape = 100 * np.mean(errors / y_train)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [58]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_train, y_train)

Model Performance
Average Error: 6785.7194 degrees.
Accuracy = 84.54%.


# Extra Tree Classifier

In [59]:
from sklearn.ensemble import ExtraTreesClassifier

In [60]:
etc = ExtraTreesClassifier()

In [61]:
etc.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [62]:
etc.score(X_test, y_test)

0.9206349206349206

In [63]:
# grid search

In [64]:
param_grid={
        'n_estimators': range(50,126,25),
        'max_features': range(50,401,50),
        'min_samples_leaf': range(20,50,5),
        'min_samples_split': range(15,36,5),
    }

In [65]:
gcv = GridSearchCV(estimator = etc, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [66]:
gcv.fit(X_train, y_train)

Fitting 3 folds for each of 960 candidates, totalling 2880 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  70 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done 718 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1284 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 2014 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 2880 out of 2880 | elapsed:  2.4min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0,
                                            class_weight=None, criterion='gini',
                                            max_depth=None, max_features='auto',
                                            max_leaf_nodes=None,
                                            max_samples=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=100, n_jobs=None,
                                            oob_score=False, random_state=None,
                                            verbose=0, warm_start=False),
             iid='deprecate

In [67]:
gcv.best_estimator_

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features=50,
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=20, min_samples_split=15,
                     min_weight_fraction_leaf=0.0, n_estimators=125,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [68]:
best_grid = gcv.best_estimator_
grid_accuracy = evaluate(best_grid, X_train, y_train)

Model Performance
Average Error: 6272.4719 degrees.
Accuracy = 86.34%.


# DecisionTreeClassifier

In [69]:
from sklearn.tree import DecisionTreeClassifier

In [70]:
dtc = DecisionTreeClassifier()

In [71]:
dtc.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [72]:
dtc.score(X_test, y_test)

0.9206349206349206

In [73]:
# Grid Search 

In [74]:
parameters={'min_samples_split' : range(10,500,20),'max_depth': range(1,20,2), 'criterion' : ['gini', 'entropy']}

In [75]:
gs_DTC = GridSearchCV(estimator = etc, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 1)

In [76]:
gs_DTC.fit(X_train, y_train)

Fitting 3 folds for each of 960 candidates, totalling 2880 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 388 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-1)]: Done 888 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1588 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 2488 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 2880 out of 2880 | elapsed:  2.3min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0,
                                            class_weight=None, criterion='gini',
                                            max_depth=None, max_features='auto',
                                            max_leaf_nodes=None,
                                            max_samples=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=100, n_jobs=None,
                                            oob_score=False, random_state=None,
                                            verbose=0, warm_start=False),
             iid='deprecate

In [77]:
gs_DTC.best_estimator_

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features=50,
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=20, min_samples_split=30,
                     min_weight_fraction_leaf=0.0, n_estimators=75, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [78]:
best_grid = gs_DTC.best_estimator_
grid_accuracy = evaluate(best_grid, X_train, y_train)

Model Performance
Average Error: 6846.9101 degrees.
Accuracy = 85.49%.


# GradientBoostingClassifier

In [16]:
from sklearn.ensemble import GradientBoostingClassifier

In [17]:
gbc = GradientBoostingClassifier()

In [44]:
gbc.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [45]:
gbc.score(X_test, y_test)

0.9365079365079365

In [29]:
param_grid = {
    "learning_rate": [0.01, 0.025],
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "subsample":[0.5, 0.8, 1.0],
    "n_estimators":[100, 200, 300, 1000]
    }

In [30]:
GBC_gscv = GridSearchCV(gbc, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [31]:
GBC_gscv.fit(X_train, y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 37.5min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed: 53.2min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_c...e,
               

In [33]:
GBC_gscv.best_estimator_

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.025, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=300,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=0.8, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [32]:
best_grid = GBC_gscv.best_estimator_
grid_accuracy = evaluate(best_grid, X_train, y_train)

Model Performance
Average Error: off by $192.4157 .
Accuracy = 99.52%.


# GradientBoostingRegressor

In [29]:
from sklearn.ensemble import GradientBoostingRegressor

In [30]:
gbr = GradientBoostingRegressor()

In [31]:
gbr.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [32]:
gbr.score(X_test, y_test)

0.6243746658982627

In [37]:
param_grid={'n_estimators':[100, 200, 300, 500, 800, 1000, 1200], 'learning_rate': [0.1, 0.05, 0.01], 'max_depth':[6,4,6], 'min_samples_leaf':[3,5,9,12]} 

In [38]:
gbr_gs = GridSearchCV(gbr, param_grid, cv=10, n_jobs=-1)

In [39]:
gbr_gs.fit(X_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_...
                            

In [41]:
gbr_gs.best_estimator_

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=3, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [40]:
best_grid = gbr_gs.best_estimator_
grid_accuracy = evaluate(best_grid, X_train, y_train)

Model Performance
Average Error: off by $603.8886 .
Accuracy = 98.47%.


# Saving model for deployment

In [ ]:
import pickle
with open('Glassdoor_Production_Model', 'wb') as f:
          pickle.dump(gbr_gs,f)

In [ ]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open('columns.json','w') as f:
    f.write(json.dumps(columns))